# 0.0 Imports

In [1]:
%load_ext autoreload
%autoreload 2

#this allows to change external without restarting kernel

In [2]:
from astropy.io import fits
import os

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
import sklearn.decomposition
import seaborn as sn
import pickle
from sklearn.preprocessing import MinMaxScaler, RobustScaler, normalize
if("Analysis" in os.getcwd()):
    os.chdir("../")

from Data_handling.pca_functions import Pca_Fitting

from Data_handling.vae import Encoder,Decoder, VAE , loss_function, model_train, print_f


os.chdir("Analysis")
os.getcwd()

'/Users/mattocallaghan/Documents/GES/Analysis'

# 1.0 Data input for HR10 Sample

THE HR10 Giraffe spectra have been selected. We will use the data handling function to create a list of these
spectra with their object names. We will get a list of spectrum classes.


In [3]:
os.chdir('../')
from Data_handling.spectra_object import get_data
os.chdir('Parameter_files')
HR10=pd.read_csv('HR10_parameter_with_file_names_for_spectra.csv')
os.chdir('../Data')
HR10['spectrum']=HR10['hr10_file'].apply(get_data)

In [ ]:
try:
    HR10=HR10[HR10['spectrum'].notna()].reset_index()
except:
    print('except')

except


In [ ]:
HR10.head()

,level_0,Unnamed: 0,index,CNAME,CNAME_iDR6_orig,GAIA_eDR3_ID,DIST_GAIA,GES_FLD,OBJECT,FILENAME,...,SYS_ERR_FEH,XI,E_XI,VRAD,E_VRAD,VRAD_OFFSET,GAMMA,E_GAMMA,hr10_file,spectrum
0,0,0,1,00000010-5727519,00000010-5727519,4919430806123101568,0.0453,GES_MW_00_01,G_2_b_566_225,gir_00000010-5727519_H548.8.fit|gir_00000010-5...,...,NaN,NaN,NaN,24.66,0.33,NaN,NaN,NaN,gir_00000010-5727519_H548.8.fit,wavelength flux flux_err 0 ...
1,1,1,2,00000014-6003143,00000014-6003143,4918001166129129856,0.3006,GES_MW_00_01,G_2_r_353_228,gir_00000014-6003143_H548.8.fit|gir_00000014-6...,...,NaN,NaN,NaN,-21.32,0.11,NaN,NaN,NaN,gir_00000014-6003143_H548.8.fit,wavelength flux flux_err 0 ...
2,2,2,3,00000064-5739305,00000064-5739305,4919413213937060992,0.0554,GES_MW_00_01,G_2_r_566_162,gir_00000064-5739305_H548.8.fit|gir_00000064-5...,...,NaN,NaN,NaN,-16.93,0.12,NaN,NaN,NaN,gir_00000064-5739305_H548.8.fit,wavelength flux flux_err 0 ...
3,3,3,4,00000097-5946444,00000097-5946444,4918020201424204288,0.2127,GES_MW_00_01,G_2_e_353_353,gir_00000097-5946444_H548.8.fit|gir_00000097-5...,...,NaN,NaN,NaN,19.05,0.54,NaN,NaN,NaN,gir_00000097-5946444_H548.8.fit,wavelength flux flux_err 0 5...
4,4,4,5,00000117-5951314,00000117-5951314,4918018754019017088,0.3639,GES_MW_00_01,G_2_e_353_319,gir_00000117-5951314_H548.8.fit|gir_00000117-5...,...,NaN,NaN,NaN,-65.99,0.39,NaN,NaN,NaN,gir_00000117-5951314_H548.8.fit,wavelength flux flux_err 0 ...


# Learning spectrum

In [ ]:
learning_spectra=[]
for i in range(0,len(HR10['spectrum'])):
    learning_spectra.append(HR10['spectrum'][i]['flux'])
    
learning_spectra=np.array(learning_spectra)
np.shape(learning_spectra)    

(49625, 5541)

# Normalise the data

In [ ]:
def normalised_data(list_spec): 
    X_train_scaled = normalize(list_spec)
    return X_train_scaled
learning_spectra=normalised_data(learning_spectra)

# 3.0 Variational Autoencoders.

In [ ]:
import torch
import torch.nn as nn 
import torch.nn.functional as F 
from torch.optim import Adam
from torch.utils.data import DataLoader


DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# 3.0.1 Set Model Hyperparameters

In [ ]:
batch_size = 1985 #factor for 1090
lr = 1e-3
epochs = 100

# 3.1 Load in the Data

In [ ]:
vae_spec=DataLoader(torch.tensor(learning_spectra),batch_size=batch_size) 

# 3.2 Model Setup

## 3.2.1 Set latent variables dimension


In [ ]:
zdim=10

# 3.2.2 Define the VAE Model and Optimiser

In [ ]:

encoder = Encoder(input_dim=learning_spectra.shape[1], hidden_dim=5, z_dim=zdim)
decoder = Decoder(output_dim=learning_spectra.shape[1], hidden_dim = 5, z_dim=zdim)
model = VAE(Encoder=encoder, Decoder=decoder).to(DEVICE)
optimizer=Adam(model.parameters(),lr=lr)

TypeError: super(type, obj): obj must be an instance or subtype of type

# 3.3 Train model 

In [ ]:
print("Start training VAE...")
model.train()
model_train(vae_spec=vae_spec,model=model,optimizer=optimizer,loss_function=loss_function,epochs=epochs,batch_size=batch_size)

Start training VAE...
	Epoch 1 complete! 	Average Loss:  4118.378159109991
	Epoch 2 complete! 	Average Loss:  4008.7330289672545


KeyboardInterrupt: 

# 3.4 Reconstruction Accuracy

In [ ]:
model.eval()
x_recon=model(torch.tensor(learning_spectra))[0].detach().numpy()

AttributeError: 'DataLoader' object has no attribute 'size'

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(x_recon[0],label='VAE Vanilla')
plt.plot(x_recon[200],label='VAE Vanilla')
plt.plot(x_recon[500],label='VAE Vanilla')


plt.legend()
plt.title('Plot of VAE reconstructed spectrum')
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(pca.learning_spectra[0],label='original')
plt.plot(pca.reconstructed_spectrum[0],label='pca',color='r',alpha=0.8)
plt.plot(x_recon[0],label='VAE Vanilla',color='green',alpha=0.7)
plt.ylim(0.005,0.020)
plt.title('Plot of original, pca and VAE reconstructed spectrum')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(pca.pca.mean_ ,label='mean from PCA',alpha=0.8)
plt.plot(x_recon[0],label='VAE Vanilla')
#plt.ylim(0.005,0.020)
plt.title('Plot of mean spectrum from PCA vs VAE reconstructed spectrum')
plt.legend()
plt.show()

# 3.5 Analysis of Latent Parameters